In [1]:
%gui qt5

import time
import numpy as np
import queue
#import yappi # needed for benchmarking multithreaded code

import napari
from napari.qt import thread_worker
from magicgui import magicgui
import tifffile
import logging
import numpy as np
from pyqtgraph import PlotWidget, PlotDataItem

In [2]:
# img_queue = queue.Queue()cmap = ['plasma']
sleep_time = 0.01
acq_running = False
start = 0
end = 0

In [3]:
plot_widget = None
plot_data = []

In [4]:
def display_napari(args):
    global start
    global end
    
    frame_ix, frame = args
    
    if frame is None:
        end = time.time()
        print(f'framerate: {1 / ((end - start) / 1309)}')        
        return
    
#     fit_onacid(frame)
    
    if len(viewer.layers) < 1:
        start = time.time()
        viewer.add_image(frame, colormap='magma', contrast_limits=[0,10])
        set_components()
    
    else:
        if frame_ix % 5 == 0: #display every 10th frame
            layer = viewer.layers[0]
            layer.data = frame
        update_components()

def set_components():
    global plot_widget
    global plot_data
    
    plot_widget = PlotWidget()
    plot_data = []
    
    for i, c in enumerate(np.random.rand(50, 2000)):
        plot_data.append(PlotDataItem(c + i))
        plot_widget.addItem(plot_data[-1])
    
    plot_widget.show()
        
def update_components():
    global plot_data
    
    for i, c in enumerate(np.random.rand(50, 2000)):
        plot_data[i].setData(c + i)

@thread_worker
def fit_onacid(frame):
    pass
    
@thread_worker(connect={'yielded': display_napari})
def yield_img(sampling_rate=100):
    """ 
    Worker thread that checks whether there are elements in the
    queue, reads them out.
    Connected to display_napari function to update display
    """
    tif = tifffile.TiffFile('/home/kushal/repos/zfish.tiff')
    
    for i in range(tif.shaped_metadata[0]['shape'][0]):
        time.sleep(1 / sampling_rate)
        frame = tif.asarray(key=i, maxworkers=8)
        yield (i, frame)
    
    
    yield (None, None)

In [5]:
viewer = napari.Viewer()

2021-07-12 02:44:21.099560: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [6]:
yield_img(sampling_rate=100)

framerate: 89.77897875303437


In [28]:
img = tifffile.imread('/home/kushal/repos/zfish.tiff')

In [29]:
img.shape

(1309, 512, 512)